In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
## Load the ANN trained model and scaler pickle, onehot
reg_model = load_model('salary_prediction__reg_model.h5')

## Load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler_regression.pkl', 'rb') as file:
    reg_scaler = pickle.load(file)

In [3]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'Exited': 0
}

In [4]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df = pd.DataFrame([input_data]) 
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,600,France,Male,40,3,60000,2,1,1,0


In [6]:
## Encode categorical values
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,600,France,1,40,3,60000,2,1,1,0


In [7]:
input_df = pd.concat([input_df.drop('Geography', axis=1).reset_index(drop=True), geo_encoded_df.reset_index(drop=True)],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,0,1.0,0.0,0.0


In [8]:
input_scaled = reg_scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.50857963,  1.00150113,
        -0.57946723, -0.57638802]])

In [10]:
prediction = reg_model.predict(input_scaled)
print("Estimated Salary:", prediction[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Estimated Salary: 90790.56
